In [6]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pandas as pd
import numpy as np
import Stats_Collector as sc

#Move indicator into its own Class so that will show up in output file. 
#Reference: https://ww，w.backtrader.com/docu/inddev.html
class SMA_Crossup_5_30(bt.Indicator):
    lines = ("sma_up_5_30",)
    def __init__(self):
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data,period=5)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data,period=30)
        self.lines.sma_up_5_30 = bt.indicators.CrossUp(self.sma_short, self.sma_long)
        
class SMA_Crossdown_5_30(bt.Indicator):
    lines = ("sma_down_5_30",)
    def __init__(self):
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data,period=5)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data,period=30)
        self.lines.sma_down_5_30 = bt.indicators.CrossDown(self.sma_short, self.sma_long)
class SMA_Crossup_50_200(bt.Indicator):
    lines = ("sma_up_50_200",)
    def __init__(self):
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data,period=50)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data,period=200)
        self.lines.sma_up_50_200 = bt.indicators.CrossUp(self.sma_short, self.sma_long)
        
class SMA_Crossdown_50_200(bt.Indicator):
    lines = ("sma_down_50_200",)
    def __init__(self):
        self.sma_short = bt.indicators.SimpleMovingAverage(self.data,period=50)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.data,period=200)
        self.lines.sma_down_50_200 = bt.indicators.CrossDown(self.sma_short, self.sma_long)

class EMA_Crossup_12_26(bt.Indicator):
    lines = ("ema_up_12_26",)
    def __init__(self):
        self.ema_short = bt.indicators.ExponentialMovingAverage(self.data,period=12)
        self.ema_long = bt.indicators.ExponentialMovingAverage(self.data,period=26)
        self.lines.ema_up_12_26 = bt.indicators.CrossUp(self.ema_short, self.ema_long)
class EMA_Crossdown_12_26(bt.Indicator):
    lines = ("ema_down_12_26",)
    def __init__(self):
        self.ema_short = bt.indicators.ExponentialMovingAverage(self.data,period=12)
        self.ema_long = bt.indicators.ExponentialMovingAverage(self.data,period=26)
        self.lines.ema_down_12_26 = bt.indicators.CrossDown(self.ema_short, self.ema_long)
class MACD_Crossup(bt.Indicator):
    lines = ("macd_up",)
    def __init__(self):
        self.macd=bt.indicators.MACD(self.data)
        self.lines.macd_up=bt.indicators.CrossUp(self.macd.macd,self.macd.signal)

class MACD_Crossdown(bt.Indicator):
    lines = ("macd_down",)
    def __init__(self):
        self.macd=bt.indicators.MACD(self.data)
        self.lines.macd_down=bt.indicators.CrossDown(self.macd.macd,self.macd.signal)
class DEMA_CrossUp_21_55(bt.Indicator):
    lines = ("dema_up_21_55",)
    def __init__(self):
        self.dema_short = bt.indicators.DoubleExponentialMovingAverage(self.data,period=21)
        self.dema_long = bt.indicators.DoubleExponentialMovingAverage(self.data,period=55)
        self.lines.dema_up_21_55 = bt.indicators.CrossUp(self.dema_short, self.dema_long)
class DEMA_CrossDown_21_55(bt.Indicator):
    lines = ("dema_down_21_55",)
    def __init__(self):
        self.dema_short = bt.indicators.DoubleExponentialMovingAverage(self.data,period=21)
        self.dema_long = bt.indicators.DoubleExponentialMovingAverage(self.data,period=55)
        self.lines.dema_down_21_55 = bt.indicators.CrossDown(self.dema_short, self.dema_long)

class PSAR_above(bt.Indicator):
    lines=("psar_above",)
    def __init__(self):
        self.lines.psar_above=bt.indicators.PSAR(self.data)>self.data
class PSAR_below(bt.Indicator):
    lines=("psar_below",)
    def __init__(self):
        self.lines.psar_below=bt.indicators.PSAR(self.data)<self.data
class ADX_up(bt.Indicator):
    lines=("adx_up",)
    
    params=(
        ('level',40),
    )
    def __init__(self):
        #print((bt.indicators.ADX(self.data)>40)[0])
        self.sma_1=bt.indicators.SimpleMovingAverage(self.data,period=1)
        self.sma_2=bt.indicators.SimpleMovingAverage(self.data,period=2)
        self.lines.adx_up=((bt.indicators.ADX(self.data)>self.p.level) + (bt.indicators.CrossUp(self.sma_1,self.sma_2)))>1
class ADX_down(bt.Indicator):
    lines=("adx_down",)
    
    params=(
        ('level',40),
    )
    def __init__(self):
        #print((bt.indicators.ADX(self.data)>40)[0])
        self.sma_1=bt.indicators.SimpleMovingAverage(self.data,period=1)
        self.sma_2=bt.indicators.SimpleMovingAverage(self.data,period=2)
        self.lines.adx_down=((bt.indicators.ADX(self.data)>self.p.level) + (bt.indicators.CrossDown(self.sma_1,self.sma_2)))>1
# Create a Strategy
class TestStrategy(bt.Strategy):
    params = (
        ("symbols", None),
        ("max_days_in_trade", 7)
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        symbols = self.p.symbols
        
        # Keep a reference to the close and forward lagged close prices as well as returns
        self.stock = self.datas[symbols.index("stockpx")]
        self.stock_px = self.stock.close
        self.stock_px_enter = self.datas[symbols.index("stockpx_lag_1")].close
        self.stock_px_exit = self.datas[symbols.index("stockpx_lag_%d" % self.p.max_days_in_trade)].close
        self.stock_returns = self.stock_px / self.stock_px(-1) - 1
        
        # Add indicators
        # Use this instead of below to be able to see full indicator (not just RSI) in output file
        self.pSAR=bt.indicators.PSAR(self.stock)
        self.sma_up_50_200=SMA_Crossup_50_200(self.stock_px)
        self.sma_down_50_200=SMA_Crossdown_50_200(self.stock_px)
        self.sma_up_5_30=SMA_Crossup_5_30(self.stock_px)
        self.sma_down_5_30=SMA_Crossdown_5_30(self.stock_px)
        self.ema_up_12_26=EMA_Crossup_12_26(self.stock_px)
        self.ema_down_12_26=EMA_Crossdown_12_26(self.stock_px)
        self.macd_up=MACD_Crossup(self.stock_px)
        self.macd_down=MACD_Crossdown(self.stock_px)
        self.dema_up_21_55=DEMA_CrossUp_21_55(self.stock_px)
        self.dema_down_21_55=DEMA_CrossDown_21_55(self.stock_px)
        self.psar_above=PSAR_above(self.stock)
        self.psar_below=PSAR_below(self.stock)
        self.adx_up=ADX_up(self.stock)
        self.adx_down=ADX_down(self.stock)
        self.in_trade = False
        self.days_in_trade = 0
        
    def next(self):
        # Simply log the closing price of the series from the reference
#         self.log('Close, %.2f' % self.stock_px[0])
        
        size = self.position.size
        
        if(not size):
            if self.ema_down_12_26[0] > 0:
                # replicate straddle payout less premium
#                 print("Indicator triggered...entering position")
#                 print('Exit price %.2f enter price %.2f' % (self.stock_px_exit[0], self.stock_px_enter[0]))
                if self.stock_px_exit[0] > self.stock_px_enter[0]:
#                     print("Exit price will be higher than enter price. Buy!")
                    self.buy(exectype=bt.Order.Close)
                else:
#                     print("Exit price will be lower than enter price. Sell!")
                    self.sell(exectype=bt.Order.Close)
                self.days_in_trade = 1
        else:
            self.days_in_trade += 1
#             print("In position %d days" % self.days_in_trade)
            if self.days_in_trade == self.p.max_days_in_trade:
#                 print("Max days to hold trade reached...closing position")
                self.close(exectype=bt.Order.Close)
                self.days_in_trade = 0
#         if(not size):
#             #if self.rsi_oversold[0] > 0:
#             if self.sma_up_50_200[0]>0:
#                 # replicate straddle payout less premium
#                 #print("Indicator triggered...entering position")
#                 #print('Exit price %.2f enter price %.2f' % (self.stock_px_exit[0], self.stock_px_enter[0]))
#                 if self.stock_px_exit[0] > self.stock_px_enter[0]:
#                     #print("Exit price will be higher than enter price. Sell!")
#                     self.sell(exectype=bt.Order.Close)
#                 else:
#                     #print("Exit price will be lower than enter price. Buy!")
#                     self.buy(exectype=bt.Order.Close)
#                 self.days_in_trade = 1
#         else:
#             self.days_in_trade += 1
#             #print("In position %d days" % self.days_in_trade)
#             if self.days_in_trade == self.p.max_days_in_trade:
#                 #print("Max days to hold trade reached...closing position")
#                 self.close(exectype=bt.Order.Close)
#                 self.days_in_trade = 0
                
        
        
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()
    
    n_lags = 8 
    
    # Read in stock and vol data 
    datapath_vol = os.path.join('../../../datas/spx-vol-5d.csv') 
    datapath_stock = os.path.join('../../../datas/spx-1993-2018.csv')
    data_vol = pd.read_csv(datapath_vol, parse_dates=True,index_col=0)
    data_stock = pd.read_csv(datapath_stock, parse_dates=True,index_col=0)
    
    # Lag and merge data
    for n in np.arange(1, n_lags):
        data_stock['stockpx_lag_%d' % n ] = data_stock['Close'].shift(-n)  
    data_stock = data_stock.dropna()
    
    mergedDf = pd.concat([data_vol,data_stock], axis=1,join='inner')
    data_vol = data_vol.loc[data_vol.index & mergedDf.index]
    data_stock = data_stock.loc[data_stock.index & mergedDf.index]
    
    # Add stock price data
    cerebro.adddata(bt.feeds.PandasData(dataname=data_stock[['Close', 'High', 'Low']]), name='stockpx')
    
    # Add lagged stock prices to be able to replicate straddle payout
    symbol_cols = [ 'stockpx_lag_%d' % n for n in np.arange(1, n_lags)] 
    for i, symbol in enumerate(symbol_cols):
        data = data_stock[[symbol]]
        data.columns = ["Close"]
        cerebro.adddata(bt.feeds.PandasData(dataname=data), name=symbol)
    
    # Add straddle premium
    data = data_vol[['straddle']]
    data.columns = ["Close"]
    cerebro.adddata(bt.feeds.PandasData(dataname=data), name='straddle')
    
    # Add a strategy
    symbol_cols = ['stockpx'] + symbol_cols + ['straddle']
    cerebro.addstrategy(TestStrategy, symbols=symbol_cols)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='SPX-ema_down_12_26.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Final Portfolio Value: 102265.70
